In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

**Load the Dataset**

In [4]:
# Load the flowers dataset
(train_ds, val_ds), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [5]:
num_classes = dataset_info.features['label'].num_classes
print(f"Number of classes: {num_classes}")

# Define parameters
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

Number of classes: 5


**Preprocessing**

In [6]:
# Preprocess and augment data
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def augment(image, label):
    # Random left-right flip
    image = tf.image.random_flip_left_right(image)
    # Random rotation (0, 90, 180, or 270 degrees)
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    # Random brightness adjustment
    image = tf.image.random_brightness(image, max_delta=0.2)
    # Random contrast adjustment
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

In [7]:
# Set up the data pipelines
train_ds = (train_ds
    .map(preprocess_image, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE))

val_ds = (val_ds
    .map(preprocess_image, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE))

**Training**

In [8]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [9]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Train the model
EPOCHS = 2
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)

Epoch 1/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 39s 251ms/step - accuracy: 0.2831 - loss: 1.6089 - val_accuracy: 0.3719 - val_loss: 1.5020
Epoch 2/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - accuracy: 0.3350 - loss: 1.5286 - val_accuracy: 0.3351 - val_loss: 1.5059


**Testing**

In [12]:
# Save the trained model
model.save('imagenet_flowers_model.h5')

# Evaluate the model on the validation set
evaluation = model.evaluate(val_ds)
print(f"Validation Loss: {evaluation[0]}")
print(f"Validation Accuracy: {evaluation[1]}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.3651 - loss: 1.4746
Validation Loss: 1.5058674812316895
Validation Accuracy: 0.3351498544216156
